ಕೆಳಗಿನ ನೋಟ್ಬುಕ್‌ಗಳನ್ನು ಚಾಲನೆ ಮಾಡಲು, ನೀವು ಇನ್ನೂ ಮಾಡದಿದ್ದರೆ, `text-embedding-ada-002` ಅನ್ನು ಮೂಲ ಮಾದರಿಯಾಗಿ ಬಳಸುವ ಮಾದರಿಯನ್ನು ನಿಯೋಜಿಸಬೇಕು ಮತ್ತು ನಿಯೋಜನೆ ಹೆಸರನ್ನು .env ಫೈಲ್‌ನಲ್ಲಿ `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` ಎಂದು ಹೊಂದಿಸಬೇಕು.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

ಮುಂದೆ, ನಾವು ಎम्बೆಡ್ಡಿಂಗ್ ಸೂಚ್ಯಂಕವನ್ನು ಪಾಂಡಾಸ್ ಡೇಟಾಫ್ರೇಮ್‌ಗೆ ಲೋಡ್ ಮಾಡಲಿದ್ದೇವೆ. ಎम्बೆಡ್ಡಿಂಗ್ ಸೂಚ್ಯಂಕವು `embedding_index_3m.json` ಎಂಬ JSON ಫೈಲ್‌ನಲ್ಲಿ ಸಂಗ್ರಹಿಸಲಾಗಿದೆ. ಎम्बೆಡ್ಡಿಂಗ್ ಸೂಚ್ಯಂಕವು 2023 ಅಕ್ಟೋಬರ್ ಕೊನೆಯವರೆಗೆ ಪ್ರತಿಯೊಂದು YouTube ಟ್ರಾನ್ಸ್ಕ್ರಿಪ್ಟ್‌ಗಳ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ಒಳಗೊಂಡಿದೆ.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

ಮುಂದೆ, ನಾವು `get_videos` ಎಂಬ ಫಂಕ್ಷನ್ ಅನ್ನು ರಚಿಸಲಿದ್ದೇವೆ, ಇದು ಪ್ರಶ್ನೆಗೆ Embedding Index ಅನ್ನು ಹುಡುಕುತ್ತದೆ. ಫಂಕ್ಷನ್ ಪ್ರಶ್ನೆಗೆ ಅತ್ಯಂತ ಸಮಾನವಾದ ಟಾಪ್ 5 ವೀಡಿಯೊಗಳನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತದೆ. ಫಂಕ್ಷನ್ ಕೆಳಗಿನಂತೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ:

1. ಮೊದಲು, Embedding Index ನ ಒಂದು ನಕಲನ್ನು ರಚಿಸಲಾಗುತ್ತದೆ.
2. ನಂತರ, OpenAI Embedding API ಬಳಸಿ ಪ್ರಶ್ನೆಯ Embedding ಲೆಕ್ಕಿಸಲಾಗುತ್ತದೆ.
3. ನಂತರ Embedding Index ನಲ್ಲಿ `similarity` ಎಂಬ ಹೊಸ ಕಾಲಮ್ ರಚಿಸಲಾಗುತ್ತದೆ. `similarity` ಕಾಲಮ್ ಪ್ರಶ್ನೆಯ Embedding ಮತ್ತು ಪ್ರತಿ ವೀಡಿಯೊ ಸೆಗ್ಮೆಂಟ್‌ಗಾಗಿ Embedding ನಡುವಿನ ಕೋಸೈನ್ ಸಾದೃಶ್ಯವನ್ನು ಹೊಂದಿದೆ.
4. ನಂತರ, Embedding Index ಅನ್ನು `similarity` ಕಾಲಮ್ ಮೂಲಕ ಫಿಲ್ಟರ್ ಮಾಡಲಾಗುತ್ತದೆ. Embedding Index ಅನ್ನು ಕೋಸೈನ್ ಸಾದೃಶ್ಯ 0.75 ಅಥವಾ ಅದಕ್ಕಿಂತ ಹೆಚ್ಚು ಇರುವ ವೀಡಿಯೊಗಳನ್ನು ಮಾತ್ರ ಒಳಗೊಂಡಂತೆ ಫಿಲ್ಟರ್ ಮಾಡಲಾಗುತ್ತದೆ.
5. ಕೊನೆಗೆ, Embedding Index ಅನ್ನು `similarity` ಕಾಲಮ್ ಮೂಲಕ ವಿಂಗಡಿಸಿ ಟಾಪ್ 5 ವೀಡಿಯೊಗಳನ್ನು ಹಿಂತಿರುಗಿಸಲಾಗುತ್ತದೆ.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ಈ ಫಂಕ್ಷನ್ ಬಹಳ ಸರಳವಾಗಿದೆ, ಇದು ಹುಡುಕಾಟ ಪ್ರಶ್ನೆಯ ಫಲಿತಾಂಶಗಳನ್ನು ಮಾತ್ರ ಮುದ್ರಿಸುತ್ತದೆ.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ಮೊದಲು, ಎम्बೆಡ್ಡಿಂಗ್ ಸೂಚ್ಯಂಕವನ್ನು ಪಾಂಡಾಸ್ ಡೇಟಾಫ್ರೇಮ್‌ಗೆ ಲೋಡ್ ಮಾಡಲಾಗುತ್ತದೆ.
2. ನಂತರ, ಬಳಕೆದಾರನಿಗೆ ಪ್ರಶ್ನೆಯನ್ನು ನಮೂದಿಸಲು ಪ್ರಾಂಪ್ಟ್ ಮಾಡಲಾಗುತ್ತದೆ.
3. ನಂತರ `get_videos` ಫಂಕ್ಷನ್ ಅನ್ನು ಕರೆಸಿ ಪ್ರಶ್ನೆಗೆ ಎम्बೆಡ್ಡಿಂಗ್ ಸೂಚ್ಯಂಕವನ್ನು ಹುಡುಕಲಾಗುತ್ತದೆ.
4. ಕೊನೆಗೆ, ಫಲಿತಾಂಶಗಳನ್ನು ಬಳಕೆದಾರರಿಗೆ ತೋರಿಸಲು `display_results` ಫಂಕ್ಷನ್ ಅನ್ನು ಕರೆಸಲಾಗುತ್ತದೆ.
5. ನಂತರ ಬಳಕೆದಾರನಿಗೆ ಮತ್ತೊಂದು ಪ್ರಶ್ನೆಯನ್ನು ನಮೂದಿಸಲು ಪ್ರಾಂಪ್ಟ್ ಮಾಡಲಾಗುತ್ತದೆ. ಬಳಕೆದಾರನು `exit` ಎಂದು ನಮೂದಿಸುವವರೆಗೆ ಈ ಪ್ರಕ್ರಿಯೆ ಮುಂದುವರಿಯುತ್ತದೆ.

![](../../../../translated_images/kn/notebook-search.1e320b9c7fcbb0bc.png)

ನೀವು ಪ್ರಶ್ನೆಯನ್ನು ನಮೂದಿಸಲು ಪ್ರಾಂಪ್ಟ್ ಮಾಡಲಾಗುತ್ತದೆ. ಒಂದು ಪ್ರಶ್ನೆಯನ್ನು ನಮೂದಿಸಿ ಎಂಟರ್ ಒತ್ತಿ. ಆಪ್ಲಿಕೇಶನ್ ಪ್ರಶ್ನೆಗೆ ಸಂಬಂಧಿಸಿದ ವೀಡಿಯೊಗಳ ಪಟ್ಟಿಯನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತದೆ. ಪ್ರಶ್ನೆಗೆ ಉತ್ತರ ಇರುವ ವೀಡಿಯೊ ಭಾಗಕ್ಕೆ ಲಿಂಕ್ ಕೂಡಾ ಆಪ್ಲಿಕೇಶನ್ ನೀಡುತ್ತದೆ.

ಇಲ್ಲಿ ಕೆಲವು ಪ್ರಯತ್ನಿಸಲು ಪ್ರಶ್ನೆಗಳು ಇವೆ:

- ಅಜೂರ್ ಮೆಷಿನ್ ಲರ್ನಿಂಗ್ ಎಂದರೆ ಏನು?
- ಕಾಂವಲ್ಯೂಷನಲ್ ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್‌ಗಳು ಹೇಗೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತವೆ?
- ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್ ಎಂದರೆ ಏನು?
- ನಾನು ಅಜೂರ್ ಮೆಷಿನ್ ಲರ್ನಿಂಗ್ ಜೊತೆಗೆ ಜುಪೈಟರ್ ನೋಟ್ಬುಕ್‌ಗಳನ್ನು ಬಳಸಬಹುದೇ?
- ONNX ಎಂದರೆ ಏನು?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
